In [77]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import base
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

train = pd.read_csv("/content/drive/My Drive/遊び/Re不動産価格推定/train_data.csv")
test = pd.read_csv("/content/drive/My Drive/遊び/Re不動産価格推定/test_data.csv")

train.head()

,id,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,土地の形状,間口,延床面積（㎡）,建築年,建物の構造,用途,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,y
0,1,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,1,２ＬＤＫ,55,NaN,NaN,NaN,昭和59年,ＳＲＣ,NaN,住宅,NaN,NaN,NaN,商業地域,80.0,600.0,2017年第４四半期,改装済,NaN,66.0
1,2,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,5,１Ｋ,20,NaN,NaN,NaN,平成15年,ＲＣ,NaN,住宅,NaN,NaN,NaN,商業地域,80.0,500.0,2017年第４四半期,未改装,NaN,19.0
2,3,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,3,１ＬＤＫ,45,NaN,NaN,NaN,平成24年,ＲＣ,住宅,その他,NaN,NaN,NaN,商業地域,80.0,500.0,2017年第３四半期,未改装,NaN,37.0
3,4,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,5,１Ｒ,20,NaN,NaN,NaN,平成15年,ＲＣ,住宅,住宅,NaN,NaN,NaN,商業地域,80.0,500.0,2017年第３四半期,未改装,NaN,18.0
4,5,宅地(土地と建物),商業地,13101,東京都,千代田区,飯田橋,飯田橋,3,NaN,80,ほぼ台形,6.8,330,昭和61年,ＲＣ,住宅、事務所、店舗,事務所,南西,区道,8.0,商業地域,80.0,500.0,2017年第２四半期,NaN,NaN,240.0


In [94]:
# train['面積（㎡）'].astype(int)
# type(train['面積（㎡）'][0])
cat_cols = list(train.columns[train.dtypes==object])
cat_cols
# 最寄駅：距離（分）
# 間口
# 面積（㎡）
# '延床面積（㎡）'
# train.loc[train['面積（㎡）']== '2000㎡以上','面積（㎡）'] = 2000
# train.loc[train['面積（㎡）']== '5000㎡以上', '面積（㎡）'] = 5000
# train['面積（㎡）'] = train['面積（㎡）'].astype('int')
# type(train['面積（㎡）'][0])

['種類',
 '地域',
 '都道府県名',
 '市区町村名',
 '地区名',
 '最寄駅：名称',
 '最寄駅：距離（分）',
 '間取り',
 '面積（㎡）',
 '土地の形状',
 '間口',
 '延床面積（㎡）',
 '建築年',
 '建物の構造',
 '用途',
 '今後の利用目的',
 '前面道路：方位',
 '前面道路：種類',
 '都市計画',
 '取引時点',
 '改装',
 '取引の事情等']

In [79]:
class KFoldTargetEncoderTrain(base.BaseEstimator, base.TransformerMixin):
  def __init__(self, colnames, targetName, n_fold=5, verbosity=True, discardOriginal_col=False):
    self.colnames = colnames
    self.targetName = targetName
    self.n_fold = n_fold
    self.verbosity = verbosity
    self.discardOriginal_col = discardOriginal_col
  
  # モデルを返す
  def fit(self, X, y=None):
    return self
  
  def transform(self, X):
    assert(type(self.targetName) == str)
    assert(type(self.colnames) == str)
    assert(self.colnames in X.columns)
    assert(self.targetName in X.columns)

    mean_of_target = X[self.targetName].mean()
    kf = KFold(n_splits = self.n_fold, shuffle = False, random_state=2019)
    col_mean_name = self.colnames + '_' + 'KFold_Target_Enc'
    X[col_mean_name] = np.nan

    for tr_ind, val_ind in kf.split(X):
      X_tr, X_val = X.iloc[tr_ind],X.iloc[val_ind]
      X.loc[X.index[val_ind], col_mean_name] = X_val[self.colnames].map(X_tr.groupby(self.colnames)[self.targetName].mean())
      X[col_mean_name].fillna(mean_of_target, inplace = True)  # nanになってしまったところは平均値で埋める --(1)

    if self.verbosity:            
      encoded_feature = X[col_mean_name].values
      print('Correlation between the new feature, {} and, {} is {}.'.format(col_mean_name,self.targetName, 
                                                                                  np.corrcoef(X[self.targetName].values,encoded_feature)[0][1]))
    if self.discardOriginal_col:
      X = X.drop(self.targetName, axis=1)
    return X
class TargetEncoderTest(base.BaseEstimator, base.TransformerMixin):
  """How to use.
  test_targetc = TargetEncoderTest(new_train,
                                   'Feature',
                                    'Feature_Kfold_Target_Enc')
  new_test = test_targetc.fit_transform(test)
  """

  def __init__(self,train,colNames,encodedName):

      self.train = train
      self.colNames = colNames
      self.encodedName = encodedName

  def fit(self, X, y=None):
      return self

  def transform(self,X):       
      mean =  self.train[[self.colNames, self.encodedName]].groupby(self.colNames).mean().reset_index() 

      dd = {}
      for index, row in mean.iterrows():
          dd[row[self.colNames]] = row[self.encodedName]
          X[self.encodedName] = X[self.colNames]
      X = X.replace({self.encodedName: dd})
      return X

treadtimeとstation_name

In [97]:
def preprocessor(train,test):
  # カテゴリ変数を数値に変換

  # 日本語変数名を英語変数名に
  En_namelist = ['id', 'build_type', 'area', 'area_code', 'prefectures', 'city', 'district', 'station_name', 'station_dist', 'floor_plan', 'floor', 'shape', 'frontage', 'total_floor', 'age', 'structure',
                'usage', 'purpose', 'road_azimuth', 'road_type', 'road_width', 'cityplan', 'coverage', 'FloorArea_ratio', 'treadtime', 'renovation', 'tread_circumstance', 'y']
  Ja_namelist = list(train.columns)
  d = dict(zip(Ja_namelist, En_namelist))
  train = train.rename(columns=d)
  del d["y"]
  test = test.rename(columns=d)

  # いらない変数をドロップ
  # 'id'
  train = train.drop('id',axis=1)
  test = test.drop('id',axis=1)

  df = pd.concat([train, test])
  # 一部文字列が含まれていてstrになっていた変数をint型に変更
  # station_dist, frontage, floor, total_floor
  df.loc[df['station_dist']== '2H?', 'station_dist'] = 60*2
  df.loc[df['station_dist']== '1H30?2H', 'station_dist'] = 60+45
  df.loc[df['station_dist']== '1H?1H30', 'station_dist']= 60+15
  df.loc[df['station_dist']== '30分?60分', 'station_dist'] = 45

  df.loc[df['frontage']== '50.0m以上', 'frontage'] = 50.0

  df.loc[df['floor']== '2000㎡以上', 'floor'] = 2000
  df.loc[df['floor']== '5000㎡以上', 'floor'] = 5000
  

  df.loc[df['total_floor']== '2000㎡以上', 'total_floor'] = 2000
  df.loc[df['total_floor']== '10m^2未満', 'total_floor'] = 9


  # df['floor'].fillna(str(df['floor'].mean())) 
  # df['frontage'].fillna(str(df['frontage'].mean()))
  # df['station_dist'].fillna(str(df['station_dist'].mean()))
  # df['total_floor'].fillna(str(df['total_floor'].mean()))
  df['floor'] = df['floor'].astype('float')
  df['frontage'] = df['frontage'].astype('float')
  df['station_dist'] = df['station_dist'].astype('float')
  df['total_floor'] = df['total_floor'].astype('float')


  train,test = make_features(df,train)

  cat_cols = list(train.columns[train.dtypes==object])
  X_train = train.drop(cat_cols, axis=1)
  X_train = X_train.drop("y", axis=1)
  y_train = np.log1p(train['y']) 
  test = test.drop(cat_cols, axis=1)
  print(test)

  # 標準化
  # X_train,test = SS(X_train,test)

  return X_train, y_train, test
def SS(X_train,test):
  scaler = StandardScaler()
  scaler.fit(X_train)
  scaler.transform(X_train)
  X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
  scaler = StandardScaler()
  scaler.fit(test)
  scaler.transform(test)
  test = pd.DataFrame(scaler.transform(test), columns=test.columns)
  return X_train, test

特徴量作成

In [98]:
def make_features(df,train):
  
  ## 特徴量作成
  # 建築年(age)
  df[df['age']=='戦前']['num_age'] = 0
  df = df.fillna({'age': 100})
  for i in range(21,64+31):
    df[df['age']=='昭和{}年'.format(i)]['num_age'] = i
  del df['age']

  # floorとtotal_floorの変数
  df['floor_total_floor'] = df['floor']/df['total_floor']
  df['floor2'] = df['floor']**2
  df['total_floor2'] = df['total_floor']**2
  df['logfloor'] = np.log(df['floor']+1)
  df['logtotal_floor'] = np.log(df['total_floor'])
  # 'coverage', 'FloorArea_ratio'建蔽率と容積率の変数
  # 建蔽率 = 建築面積/敷地面積*100
  # 容積率 = 延床面積/敷地面積*100
  df['Buildingarea'] = df['floor']*df['coverage']/100
  df['build_total'] = df['coverage']/df['FloorArea_ratio']
  df['cverage2'] = df['coverage']**2
  df['FloorArea_ratio2'] = df['FloorArea_ratio']**2

  # station_dist
  df['logstation_dist'] = np.log(df['station_dist'])


  ## エンコーディングによる特徴量作成
  
  #LabelEncoder
  for i in ['station_name','treadtime','structure','usage']:
    df = df.fillna({i: '欠損値'})
    le = LabelEncoder()
    encoded = le.fit_transform(df[i])
    df['LE_{}'.format(i)] = encoded
    del df[i]
  train = df.iloc[: len(train), :]
  test = df.iloc[len(train): , :]
  del test['y']
  # kFoldTargetEncoding 参考リンク=https://qiita.com/ground0state/items/6778dc2132abdabaf914
  cat_cols = list(train.columns[train.dtypes==object])
  for colname in cat_cols:
    targetc = KFoldTargetEncoderTrain(colname,'y',n_fold=5)
    train = targetc.fit_transform(train)
    test_targetc = TargetEncoderTest(train, colname, '{}_KFold_Target_Enc'.format(colname))
    test = test_targetc.fit_transform(test)

  return train, test
def LOF(dfs):
  clf = LocalOutlierFactor(n_neighbors=8, contamination=0.01)
  pred = clf.fit_predict(dfs)

In [99]:
X_train,y_train,test = preprocessor(train,test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an 

Correlation between the new feature, build_type_KFold_Target_Enc and, y is 0.07706653891425425.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Correlation between the new feature, area_KFold_Target_Enc and, y is 0.1859880841612339.
Correlation between the new feature, prefectures_KFold_Target_Enc and, y is -0.04705246355958163.
Correlation between the new feature, city_KFold_Target_Enc and, y is 0.09510476850601315.
Correlation between the new feature, district_KFold_Target_Enc and, y is 0.12453392325509882.
Correlation between the new feature, floor_plan_KFold_Target_Enc and, y is 0.0960623039836554.
Correlation between the new feature, shape_KFold_Target_Enc and, y is 0.07256676865510782.
Correlation between the new feature, purpose_KFold_Target_Enc and, y is 0.03708076892305165.
Correlation between the new feature, road_azimuth_KFold_Target_Enc and, y is 0.0564233728109305.
Correlation between the new feature, road_type_KFold_Target_Enc and, y is 0.14305753123534923.
Correlation between the new feature, cityplan_KFold_Target_Enc and, y is 0.05889704698011904.
Correlation between the new feature, renovation_KFold_Target_Enc

In [100]:
# sns.pairplot(train)
X_train

,area_code,station_dist,floor,frontage,total_floor,road_width,coverage,FloorArea_ratio,floor_total_floor,floor2,total_floor2,logfloor,logtotal_floor,Buildingarea,build_total,cverage2,FloorArea_ratio2,logstation_dist,LE_station_name,LE_treadtime,LE_structure,LE_usage,build_type_KFold_Target_Enc,area_KFold_Target_Enc,prefectures_KFold_Target_Enc,city_KFold_Target_Enc,district_KFold_Target_Enc,floor_plan_KFold_Target_Enc,shape_KFold_Target_Enc,purpose_KFold_Target_Enc,road_azimuth_KFold_Target_Enc,road_type_KFold_Target_Enc,cityplan_KFold_Target_Enc,renovation_KFold_Target_Enc,tread_circumstance_KFold_Target_Enc
0,13101,1.0,55.0,NaN,NaN,NaN,80.0,600.0,NaN,3025.0,NaN,4.025352,NaN,44.0,0.133333,6400.0,360000.0,0.000000,628,49,23,201,27.811706,65.434766,59.886263,310.916854,150.333333,35.163828,65.434766,40.519363,65.434766,65.434766,95.974488,26.227973,65.434766
1,13101,5.0,20.0,NaN,NaN,NaN,80.0,500.0,NaN,400.0,NaN,3.044522,NaN,16.0,0.160000,6400.0,250000.0,1.609438,628,49,13,201,27.811706,65.434766,59.886263,310.916854,150.333333,15.836031,65.434766,40.519363,65.434766,65.434766,95.974488,28.653058,65.434766
2,13101,3.0,45.0,NaN,NaN,NaN,80.0,500.0,NaN,2025.0,NaN,3.828641,NaN,36.0,0.160000,6400.0,250000.0,1.098612,628,48,13,27,27.811706,65.434766,59.886263,310.916854,150.333333,31.032322,65.434766,68.251611,65.434766,65.434766,95.974488,28.653058,65.434766
3,13101,5.0,20.0,NaN,NaN,NaN,80.0,500.0,NaN,400.0,NaN,3.044522,NaN,16.0,0.160000,6400.0,250000.0,1.609438,628,48,13,27,27.811706,65.434766,59.886263,310.916854,150.333333,10.579357,65.434766,40.519363,65.434766,65.434766,95.974488,28.653058,65.434766
4,13101,3.0,80.0,6.8,330.0,8.0,80.0,500.0,0.242424,6400.0,108900.0,4.394449,5.799093,64.0,0.160000,6400.0,250000.0,1.098612,628,47,13,44,80.752643,261.619465,59.886263,310.916854,150.333333,65.434766,87.436089,107.137760,88.577290,107.774710,95.974488,65.434766,65.434766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356339,13421,NaN,500.0,NaN,NaN,NaN,70.0,200.0,NaN,250000.0,NaN,6.216606,NaN,350.0,0.350000,4900.0,40000.0,NaN,378,6,5,201,97.406793,55.356350,70.256298,10.298333,14.072727,65.434766,95.396057,65.434766,26.027798,65.434766,6.299835,65.434766,65.434766
356340,13421,NaN,115.0,10.0,NaN,5.5,70.0,200.0,NaN,13225.0,NaN,4.753590,NaN,80.5,0.350000,4900.0,40000.0,NaN,378,6,5,201,97.406793,55.356350,70.256298,10.298333,14.072727,65.434766,155.154449,65.434766,103.183392,70.392609,6.299835,65.434766,65.434766
356341,13421,NaN,230.0,15.0,NaN,4.8,70.0,200.0,NaN,52900.0,NaN,5.442418,NaN,161.0,0.350000,4900.0,40000.0,NaN,378,12,5,201,97.406793,74.327440,70.256298,10.298333,4.367143,65.434766,95.396057,65.434766,131.424777,70.392609,6.299835,65.434766,109.249681
356342,13421,NaN,175.0,NaN,NaN,NaN,70.0,200.0,NaN,30625.0,NaN,5.170484,NaN,122.5,0.350000,4900.0,40000.0,NaN,378,9,5,201,97.406793,55.356350,70.256298,10.298333,4.367143,65.434766,95.396057,65.434766,26.027798,65.434766,6.299835,65.434766,109.249681


In [101]:
!pip install optuna

In [106]:
import optuna.integration.lightgbm as lgb
#rmse_scores = []
#pred_cv = np.zeros(len(test_x.index))
def LGB_Model_Opt(train_x,train_y,test_x,seed):
  rmse_scores = []
  preds = []
  preds_test = []
  va_idxes = []
  pred_cv = np.zeros(len(test_x.index))


  fold_num = 20
  kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
  for n_fold, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
      tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
      tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
      lgb_train = lgb.Dataset(tr_x,tr_y)
      lgb_eval = lgb.Dataset(va_x,va_y)

      # params = {
      #     'obejective':'rmse',
      #     'seed':seed,
      #     'verbose':1,
      #     'metrics':'rmse',
      #     'max_depth' : 30,
      #     #'num_leaves' : 20 # 付け足した5/01
      # }

      # lgb_model = lgb.train(
      #     params,
      #     lgb_train,
      #     num_boost_round = 20000,
      #     valid_names=['train','valid'],
      #     valid_sets=[lgb_train,lgb_eval],
      #     verbose_eval= 1000,
      #     early_stopping_rounds=10000
      #               )
      params = {
    'objective': 'rmse',
    'metric': 'rmse'
    }

      best_params, tuning_history = dict(), list()

      lgb_model = lgb.train(
          params, lgb_train, valid_sets=[lgb_train,lgb_eval],
          verbose_eval=0
          # best_params=best_params,
          #tuning_history=tuning_history
                    )

      va_pred = lgb_model.predict(va_x)
      mse = mean_squared_error(va_y, va_pred)
      rmse = np.sqrt(mse)
      rmse_scores.append(rmse)
      
      submission = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration) 
      pred_cv += submission/fold_num
      print(n_fold+1,"回目のrmsle:",rmse)

  print("LGB rmsle平均:",np.mean(rmse_scores))
  return pred_cv
import lightgbm as lgb
def LGB_Model(train_x,train_y,test_x,seed):
  rmse_scores = []
  preds = []
  preds_test = []
  va_idxes = []
  pred_cv = np.zeros(len(test_x.index))


  fold_num = 20
  kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
  for n_fold, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
      tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
      tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
      lgb_train = lgb.Dataset(tr_x,tr_y)
      lgb_eval = lgb.Dataset(va_x,va_y)

      params = {
          'obejective':'rmse',
          'seed':seed,
          'verbose':1,
          'metrics':'rmse'
          # 'max_depth' : 100,
          # 'num_leaves' : 200
      }

      lgb_model = lgb.train(
          params,
          lgb_train,
          num_boost_round = 10000,
          valid_names=['train','valid'],
          valid_sets=[lgb_train,lgb_eval],
          verbose_eval= 1000,
          early_stopping_rounds=1000
                    )

      va_pred = lgb_model.predict(va_x)
      mse = mean_squared_error(va_y, va_pred)
      rmse = np.sqrt(mse)
      rmse_scores.append(rmse)
      
      submission = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration) 
      pred_cv += submission/fold_num
      print(n_fold+1,"回目のrmsle:",rmse)

  print("LGB rmsle平均:",np.mean(rmse_scores))
  return pred_cv

In [ ]:
#pred_cv = LGB_Model_Opt(X_train,y_train,test,71)
pred_cv = LGB_Model(X_train,y_train,test,71)

Training until validation scores don't improve for 1000 rounds.
[1000]	train's rmse: 0.2951	valid's rmse: 0.315578


In [64]:
pred_cv = np.expm1(pred_cv)
submit_df = pd.DataFrame({'y': pred_cv})
submit_df.index.name = 'id'
submit_df.index = submit_df.index + 1
submit_df.to_csv('submission_1.csv')